In [134]:
import mysql.connector
from mysql.connector import Error
import json
import re
import pandas as pd
from datetime import datetime
from DBHelper import CDBHelper
from datetime import datetime, timedelta
from ClickUpDB import CClickUpDB
import os

In [135]:
list_id = "901600183071"  # Replace with your actual ListID
start_date = "01-08-2024"  # Replace with your desired start date
end_date = "01-09-2024"    # Replace with your desired end date

# Fetch tasks based on the criteria
tasks = CClickUpDB.MSGetTasksByListID(list_id, start_date, end_date)
print(len(tasks))

Successfully connected to MySQL server
Retrieved 54 task(s) from ListID 901600183071 between 01-08-2024 and 01-09-2024.
54


In [136]:
def MSCreateDataframe(lsTasks):
    # Creating a DataFrame from the list of tasks
    df = pd.DataFrame(lsTasks)
    
    # Parsing JSON strings in 'TaskPriority' and 'TaskAssigneesList'
    df['TaskPriority'] = df['TaskPriority'].apply(lambda x: json.loads(x) if x != 'null' else None)
    df['TaskAssigneesList'] = df['TaskAssigneesList'].apply(json.loads)
    df['TaskScore'] = df.apply(CClickUpDB.calculate_task_score, include_toughness=False, axis=1)
    
    df = df[df['EstimatedTime'].apply(CClickUpDB.isEstimatedTimeProvided)]

    # Convert 'TaskStartDate' to datetime for sorting
    # df['TaskStartDate'] = pd.to_datetime(df['TaskStartDate'], format='%d-%m-%Y')

    # Sort tasks by 'TaskStartDate' ascending, then by 'TaskScore' ascending within the same date
    # df = df.sort_values(by=['TaskStartDate', 'TaskScore'])

    # Drop columns except the specified list
    columns_to_keep = ['ListName', 'TaskID', 'TaskSubject', 'TaskStartDate', 'TaskDueDate', 'TaskStatus',
                       'EstimatedTime', 'TaskPriority', 'TaskAssigneesList', 
                       'TaskIsMilestone', 'TaskIntensity', 'TaskScore','TaskCreatedDate']
    df = df[columns_to_keep]
    print(df)
    # Add new columns
    # 1. 'EstimatedTaskMin' - sum of hours * 60 + mins
    df['EstimatedTime'] = df['EstimatedTime'].apply(json.loads)
    df['EstimatedTaskMin'] = df['EstimatedTime'].apply(lambda x: (x['hrs'] * 60 + x['mins']) if x else 0)

    # 2. 'AssignTo' - extract username from the first entry in 'TaskAssigneesList'
    df['AssignTo'] = df['TaskAssigneesList'].apply(lambda x: x[0]['username'] if x and len(x) > 0 else None)

    # 3. 'TaskPriority' - extract priority
    df['TaskPriority'] = df['TaskPriority'].apply(lambda x: x['priority'] if x else None)

    # 4. 'TaskStatus' - assuming there's a 'TaskStatus' field structured similarly
    df['TaskStatus'] = df['TaskStatus'].apply(json.loads)
    df['TaskStatus'] = df['TaskStatus'].apply(lambda x: x['status'] if isinstance(x, dict) else None)

    # Creating employee-wise task lists
    employee_tasks = {}
    for _, row in df.iterrows():
        assignees = row['TaskAssigneesList']
        if assignees:
            for assignee in assignees:
                username = assignee['username']
                if username not in employee_tasks:
                    employee_tasks[username] = []
                # Convert the row to a dictionary and append to the list
                employee_tasks[username].append(row.to_dict())

    # Directory to save the Excel files
    output_directory = r"Data/"
    os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

    # Save employee-wise task lists to Excel files
    for username, tasks in employee_tasks.items():
        # Create a DataFrame from the list of task dictionaries
        df_employee = pd.DataFrame(tasks)
        # Create a filename based on the username
        filename = f"{username.replace(' ', '_')}_tasks.xlsx"
        # Construct the full path
        file_path = os.path.join(output_directory, filename)
        # Save the DataFrame to an Excel file
        df_employee.to_excel(file_path, index=False)

    print("Employee-wise task details have been saved to the 'Data/' directory.")
    
    # Return the dictionary where keys are employee names and values are lists of task dictionaries
    return employee_tasks


employee_tasks = MSCreateDataframe(tasks)
print(employee_tasks)

   ListName     TaskID                                       TaskSubject  \
3   ERPNext  86cw75tb6                         Mohit - Job Openings Form   
4   ERPNext  86cw75tdf                        Mansi - Leave Request Form   
5   ERPNext  86cw75zvc                             Mohit - Employee Form   
6   ERPNext  86cw76fjj  Mitul - Compare Odoo vs Zoho vs ERPNext Solution   
11  ERPNext  86cw7wkbw                           Restore V14 Data in V15   
15  ERPNext  86cw7wzx2                             Mitul - Review & Test   
16  ERPNext  86cw7wzy0                          Mitul - show demo to AKS   
17  ERPNext  86cw7x1eg                          Mitul - show demo to AKS   
18  ERPNext  86cw7x1eh                             Mitul - Review & Test   
19  ERPNext  86cw7x7xw                      Mansi - Import Projects List   
20  ERPNext  86cw7x7zr     Mitul - Import Customer List & Customize Form   
21  ERPNext  86cw7x82b                        Mohit  - Import Users List   
22  ERPNext 

In [141]:

import copy
def prepareTaskDateWise(employee_tasks, bDebug=True):
    dictAllocatedDateWiseTask = {}
    
    for user_name, user_tasks in employee_tasks.items():
        dictAllocatedDateWiseTask[user_name] = {}
        if bDebug:
            print("1. UserName - ", user_name)
            print("2. User Tasks - ", user_tasks)
        
        for idx, task_details in enumerate(user_tasks):
            task_start_date = task_details['TaskStartDate']
            task_id = task_details['TaskID']
            estimated_task_min = task_details['EstimatedTaskMin']
            task_end_date = task_details['TaskDueDate']
            task_details['TaskExecutionDate'] = task_start_date
            task_details['isTaskMovable'] = (task_start_date != task_end_date)
            # Check if the start date already exists in the structure
            if task_start_date not in dictAllocatedDateWiseTask[user_name]:
                # Initialize the start date entry if not present
                dictAllocatedDateWiseTask[user_name][task_start_date] = {
                    "TotalWorkMin": 8 * 60,  # Assuming 8 working hours in a day
                    "AllocatedWorkingMin": 0,  # Initialize allocated working hours
                    "IsConflict": False,  # Initialize IsConflict as False
                    "AllocatedTask": {}  # Dictionary to store task ID and detailed task data
                }
            
            # Add the task details to the 'AllocatedTask' dictionary
            dictAllocatedDateWiseTask[user_name][task_start_date]["AllocatedTask"][task_id] = task_details
            
            # Update the allocated working minutes
            dictAllocatedDateWiseTask[user_name][task_start_date]["AllocatedWorkingMin"] += estimated_task_min
            
            # Check for conflict and update IsConflict
            allocated_minutes = dictAllocatedDateWiseTask[user_name][task_start_date]["AllocatedWorkingMin"]
            total_minutes = dictAllocatedDateWiseTask[user_name][task_start_date]["TotalWorkMin"]
            dictAllocatedDateWiseTask[user_name][task_start_date]["IsConflict"] = allocated_minutes > total_minutes
            # Assume both dates are not null
            if bDebug:
                print(f"3. Task Details {idx+1} - {task_details}")
                print(f"4. Updated Structure - {dictAllocatedDateWiseTask[user_name][task_start_date]}")
    
    return dictAllocatedDateWiseTask
            
# def splitTaskPossible(dictAllocatedDateWiseTask, employee_tasks, bDebug=True):
#     for user_name, startDateTaskDetails in dictAllocatedDateWiseTask.items():
#         if bDebug:
#             print("1. UserName - ", user_name)
#             print("2. Date Wise Tasks Arrangement - ", startDateTaskDetails)

#         for taskStartDate, dateDetails in startDateTaskDetails.items():
#             splitTaskNeed = dateDetails.get("IsConflict")
            
#             if splitTaskNeed:
#                 if bDebug:
#                     print("3. Task Start Date- ", taskStartDate)
#                     print("4. Particular Date Tasks Details - ", dateDetails)

#                 # Get all task details for this date
#                 task_list = list(dateDetails.get("AllocatedTask", {}).values())

#                 # Sort tasks by TaskScore (highest to lowest) and then by TaskCreatedDate (latest to earliest)
#                 sorted_tasks = sorted(
#                     task_list,
#                     key=lambda x: (-x['TaskScore'], datetime.strptime(x['TaskCreatedDate'], '%d-%m-%Y'))
#                 )

#                 # Initialize available minutes
#                 totalTaskAvailableMins = 480  # 8 hours in minutes

#                 # Create a list to keep track of sorted task IDs
#                 task_score_order_list = []

#                 # Iterate through sorted tasks to allocate time and check conflicts
#                 for task in sorted_tasks:
#                     task_id = task['TaskID']
#                     estimated_min = task['EstimatedTaskMin']
#                     task_start_date = task['TaskStartDate']
#                     task_end_date = task['TaskDueDate']
                    
#                     # Assume both dates are not null
#                     # task['isTaskMovable'] = (task_start_date == task_end_date)
                    
#                     # Check if the task can be allocated within the available time
#                     if estimated_min <= totalTaskAvailableMins:
#                         # Mark the task as not in conflict
#                         task['IsConflict'] = False
#                         # Allocate the full estimated time
#                         task['AllocatedTimeMin'] = estimated_min
#                         task['ConflictTimeMin'] = 0
#                         # Subtract the estimated minutes from the available time
#                         totalTaskAvailableMins -= estimated_min
#                     else:
#                         # Conflict case
#                         task['IsConflict'] = True
#                         if not task['isTaskMovable']:
#                             # If totalTaskAvailableMins <= 0, no time can be allocated
#                             if totalTaskAvailableMins <= 0:
#                                 task['AllocatedTimeMin'] = 0
#                                 task['ConflictTimeMin'] = estimated_min
#                             else:
#                                 # Allocate remaining available minutes
#                                 task['AllocatedTimeMin'] = totalTaskAvailableMins
#                                 task['ConflictTimeMin'] = estimated_min - totalTaskAvailableMins
#                                 # Set totalTaskAvailableMins to 0 as it has been exhausted
#                                 totalTaskAvailableMins = 0
                        
#                     # Add the task ID to the ordered list
#                     task_score_order_list.append(task_id)

#                     if bDebug:
#                         print(f"TaskID - {task_id}, EstimatedMin - {estimated_min}, Task Movable - {task['isTaskMovable']}, IsConflict - {task['IsConflict']}")
#                         print(f"AllocatedTimeMin - {task.get('AllocatedTimeMin', 'N/A')}, ConflictTimeMin - {task.get('ConflictTimeMin', 'N/A')}")
#                         print(f"Remaining Available Minutes - {totalTaskAvailableMins}")

#                 # Update the dictionary with the ordered task IDs
#                 dateDetails['TaskScoreWiseOrder'] = task_score_order_list

#     return dictAllocatedDateWiseTask
     
def get_next_date(date_str):
    """Utility function to get the next date in 'DD-MM-YYYY' format."""
    date_obj = datetime.strptime(date_str, '%d-%m-%Y')
    next_date_obj = date_obj + timedelta(days=1)
    return next_date_obj.strftime('%d-%m-%Y')

def splitTaskPossible(dictAllocatedDateWiseTask, employee_tasks, bDebug=True):
    for user_name, startDateTaskDetails in dictAllocatedDateWiseTask.items():
        if bDebug:
            print("1. UserName - ", user_name)
            print("2. Date Wise Tasks Arrangement - ", startDateTaskDetails)

        for taskStartDate, dateDetails in list(startDateTaskDetails.items()):  # Use list to avoid runtime modification errors
            splitTaskNeed = dateDetails.get("IsConflict")
            
            if splitTaskNeed:
                if bDebug:
                    print("3. Task Start Date- ", taskStartDate)
                    print("4. Particular Date Tasks Details - ", dateDetails)

                # Get all task details for this date
                task_list = list(dateDetails.get("AllocatedTask", {}).values())

                # Sort tasks by TaskScore (highest to lowest) and then by TaskCreatedDate (latest to earliest)
                sorted_tasks = sorted(
                    task_list,
                    key=lambda x: (-x['TaskScore'], datetime.strptime(x['TaskCreatedDate'], '%d-%m-%Y'))
                )

                # Initialize available minutes
                totalTaskAvailableMins = 480  # 8 hours in minutes

                # Create a list to keep track of sorted task IDs
                task_score_order_list = []

                # Iterate through sorted tasks to allocate time and check conflicts
                for task in sorted_tasks:
                    task_id = task['TaskID']
                    estimated_min = task['EstimatedTaskMin']
                    task_start_date = task['TaskStartDate']
                    
                    # Check if the task can be allocated within the available time
                    if estimated_min <= totalTaskAvailableMins:
                        # Mark the task as not in conflict
                        task['IsConflict'] = False
                        # Allocate the full estimated time
                        task['AllocatedTimeMin'] = estimated_min
                        task['ConflictTimeMin'] = 0
                        # Subtract the estimated minutes from the available time
                        totalTaskAvailableMins -= estimated_min
                    else:

                        if not task['isTaskMovable']:
                            task['IsConflict'] = True
                            # If totalTaskAvailableMins <= 0, no time can be allocated
                            if totalTaskAvailableMins <= 0:
                                task['AllocatedTimeMin'] = 0
                                task['ConflictTimeMin'] = estimated_min
                            else:
                                # Allocate remaining available minutes
                                task['AllocatedTimeMin'] = totalTaskAvailableMins
                                task['ConflictTimeMin'] = estimated_min - totalTaskAvailableMins
                                # Set totalTaskAvailableMins to 0 as it has been exhausted
                                totalTaskAvailableMins = 0
                        # Check if the task is movable
                        else:
                            # If totalTaskAvailableMins <= 0, remove task and push to the next start date
                            if totalTaskAvailableMins <= 0:
                                task['IsConflict'] = False
                                task['AllocatedTimeMin'] = 0
                                task['ConflictTimeMin'] =0

                                # Remove the task from the current date
                                dateDetails['AllocatedTask'].pop(task_id)
                                
                                # Push to the next available date
                                next_start_date = get_next_date(task_start_date)
                                if next_start_date not in startDateTaskDetails:
                                    # Initialize the next date if it doesn't exist
                                    startDateTaskDetails[next_start_date] = {
                                        "TotalWorkMin": 480,
                                        "AllocatedWorkingMin": estimated_min,
                                        "IsConflict": estimated_min > 480,
                                        "AllocatedTask": {task_id:task}
                                    }
                                    continue
                                
                                # Add the task to the next date's task list
                                startDateTaskDetails[next_start_date]["AllocatedTask"][task_id] = task
                                startDateTaskDetails[next_start_date]["AllocatedWorkingMin"] += estimated_min
                                startDateTaskDetails[next_start_date]["IsConflict"] =  startDateTaskDetails[next_start_date]["AllocatedWorkingMin"] > startDateTaskDetails[next_start_date]["TotalWorkMin"]
                                startDateTaskDetails[next_start_date]["AllocatedTask"][task_id]['isTaskMovable'] = next_start_date != startDateTaskDetails[next_start_date]["AllocatedTask"][task_id]['TaskDueDate']
                                
                            else:
                                # Push remaining part of the task to the next date
                                remaining_min = estimated_min - totalTaskAvailableMins
                                next_start_date = get_next_date(task_start_date)
                                
                                # for current start date
                                task['IsConflict'] =  False
                                task['ConflictTimeMin'] = 0
                                task['AllocatedTimeMin'] = totalTaskAvailableMins
                                
                                # Add the remaining part of the task to the next date
                                new_task_details = task.copy()
                                new_task_details['EstimatedTaskMin'] = remaining_min
                                new_task_details['isTaskMovable'] = next_start_date != new_task_details['TaskDueDate']
                                if next_start_date not in startDateTaskDetails:
                                    new_task_details['AllocatedTimeMin'] = remaining_min if remaining_min < 480 else 480-remaining_min
                                    # Initialize the next date if it doesn't exist
                                    startDateTaskDetails[next_start_date] = {
                                        "TotalWorkMin": 480,
                                        "AllocatedWorkingMin": remaining_min,
                                        "IsConflict": remaining_min > 480,
                                        "AllocatedTask": {task_id:new_task_details}
                                    }
                                    totalTaskAvailableMins = 0
                                    continue
                                
                                availbleNextDateWorkingMin = startDateTaskDetails[next_start_date]["TotalWorkMin"] - startDateTaskDetails[next_start_date]["AllocatedWorkingMin"]
                                new_task_details['AllocatedTimeMin'] = remaining_min if remaining_min <=  availbleNextDateWorkingMin else availbleNextDateWorkingMin
                                startDateTaskDetails[next_start_date]["AllocatedTask"][task_id] = new_task_details
                                startDateTaskDetails[next_start_date]["IsConflict"] = (startDateTaskDetails[next_start_date]["AllocatedWorkingMin"] + remaining_min) > startDateTaskDetails[next_start_date]["TotalWorkMin"]
                                startDateTaskDetails[next_start_date]["AllocatedWorkingMin"] += new_task_details['AllocatedTimeMin']
                                # Set totalTaskAvailableMins to 0 as it has been exhausted
                                totalTaskAvailableMins = 0
                                    

                    # Add the task ID to the ordered list
                    task_score_order_list.append(task_id)

                    if bDebug:
                        print(f"TaskID - {task_id}, EstimatedMin - {estimated_min}, Task Movable - {task['isTaskMovable']}, IsConflict - {task['IsConflict']}")
                        print(f"AllocatedTimeMin - {task.get('AllocatedTimeMin', 'N/A')}, ConflictTimeMin - {task.get('ConflictTimeMin', 'N/A')}")
                        print(f"Remaining Available Minutes - {totalTaskAvailableMins}")

                # Update the dictionary with the ordered task IDs
                dateDetails['TaskScoreWiseOrder'] = task_score_order_list

    return dictAllocatedDateWiseTask
    
dictAllocatedDateWiseTask = prepareTaskDateWise(employee_tasks, bDebug=False)
splitTaskPossible(dictAllocatedDateWiseTask=dictAllocatedDateWiseTask,employee_tasks=employee_tasks,bDebug=True)
# print(dictAllocatedDateWiseTask)


1. UserName -  mansi solanki
2. Date Wise Tasks Arrangement -  {'20-08-2024': {'TotalWorkMin': 480, 'AllocatedWorkingMin': 300, 'IsConflict': False, 'AllocatedTask': {'86cw75tb6': {'ListName': 'ERPNext', 'TaskID': '86cw75tb6', 'TaskSubject': 'Mohit - Job Openings Form', 'TaskStartDate': '20-08-2024', 'TaskDueDate': '22-08-2024', 'TaskStatus': 'delievered', 'EstimatedTime': {'hrs': 5, 'mins': 0, 'time_estimate': 18000000}, 'TaskPriority': 'urgent', 'TaskAssigneesList': [{'id': 88895068, 'color': '#622aea', 'email': 'mansi@riveredgeanalytics.com', 'initials': 'MS', 'username': 'mansi solanki', 'profilePicture': 'https://attachments.clickup.com/profilePictures/88895068_IUP.jpg'}], 'TaskIsMilestone': 0, 'TaskIntensity': 1, 'TaskScore': 3, 'TaskCreatedDate': '12-08-2024', 'EstimatedTaskMin': 300, 'AssignTo': 'mansi solanki', 'TaskExecutionDate': '20-08-2024', 'isTaskMovable': True}}}, '21-08-2024': {'TotalWorkMin': 480, 'AllocatedWorkingMin': 960, 'IsConflict': True, 'AllocatedTask': {'86cw

{'mansi solanki': {'20-08-2024': {'TotalWorkMin': 480,
   'AllocatedWorkingMin': 300,
   'IsConflict': False,
   'AllocatedTask': {'86cw75tb6': {'ListName': 'ERPNext',
     'TaskID': '86cw75tb6',
     'TaskSubject': 'Mohit - Job Openings Form',
     'TaskStartDate': '20-08-2024',
     'TaskDueDate': '22-08-2024',
     'TaskStatus': 'delievered',
     'EstimatedTime': {'hrs': 5, 'mins': 0, 'time_estimate': 18000000},
     'TaskPriority': 'urgent',
     'TaskAssigneesList': [{'id': 88895068,
       'color': '#622aea',
       'email': 'mansi@riveredgeanalytics.com',
       'initials': 'MS',
       'username': 'mansi solanki',
       'profilePicture': 'https://attachments.clickup.com/profilePictures/88895068_IUP.jpg'}],
     'TaskIsMilestone': 0,
     'TaskIntensity': 1,
     'TaskScore': 3,
     'TaskCreatedDate': '12-08-2024',
     'EstimatedTaskMin': 300,
     'AssignTo': 'mansi solanki',
     'TaskExecutionDate': '20-08-2024',
     'isTaskMovable': True}}},
  '21-08-2024': {'TotalWorkM

In [138]:
def processCleanDF(employee_dataframes):
    # Define the maximum hours in a particular day
    max_daily_hours = 8 * 60  # 8 hours converted to minutes

    # Process each employee DataFrame
    for username, df in employee_dataframes.items():
        # Add a new column to indicate if the task fits in a particular day
        df['IsTaskFitInParticularDate'] = 'No'

        # Keep track of remaining time for each day
        remaining_time_in_minutes = max_daily_hours

        # Iterate through each task row
        for index, row in df.iterrows():
            print("row",isinstance(row['EstimatedTime'],str))
            # Get estimated hours and minutes
            
            estimated_hours = row['EstimatedTime'].get('hrs', 0)
            estimated_minutes = row['EstimatedTime'].get('mins', 0)
            total_task_minutes = estimated_hours * 60 + estimated_minutes

            # Check if the task fits within the remaining time for the day
            if total_task_minutes <= remaining_time_in_minutes:
                # Mark as "Yes" if it fits
                df.at[index, 'IsTaskFitInParticularDate'] = 'Yes'
                # Subtract the time from the remaining time for the day
                remaining_time_in_minutes -= total_task_minutes
            else:
                # If it doesn't fit, reset the remaining time for the next day
                remaining_time_in_minutes = max_daily_hours - total_task_minutes
                # Mark this task as fitting in the new day
                df.at[index, 'IsTaskFitInParticularDate'] = 'Yes'

    # Optionally, return the modified DataFrames if needed
    return employee_dataframes
        

processCleanDF(tasks)

AttributeError: 'list' object has no attribute 'items'